<a href="https://colab.research.google.com/github/dkovah/testing_stable_diffusion/blob/main/testing_stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requisitos previos

In [ ]:
%%capture
#@title 1- Instalar dependencias { display-mode: "form" }
!pip install diffusers==0.3.0
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

In [ ]:
#@title 2- Iniciar sesión en HugginFace { display-mode: "form" }
from huggingface_hub import notebook_login
from google.colab import output
import torch
from torch import autocast

output.enable_custom_widget_manager()

notebook_login()

# Generador de imágenes

In [ ]:
%%capture
#@title 1- Descargar el modelo
from PIL import Image
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True)
pipe = pipe.to("cuda")

def dummy_checker(images, **kwargs): return images, False
pipe.safety_checker = dummy_checker
from random import randint

In [ ]:
#@title 2- Generar imágenes
prompt = "" #@param {type:"string"}
inference_steps = 50 #@param {type:"number"}
guidance_scale = 10 #@param {type:"number"}
height=512 #@param {type:"number"}
width=512 #@param {type:"number"}
random_seed = True #@param {type:"boolean"}


seed = None #@param {type:"number"}

if random_seed:
  seed = randint(0, 100000000)

print("Semilla: ", seed)
generator = torch.Generator("cuda").manual_seed(seed)

with autocast("cuda"):
  image = pipe(prompt, num_inference_steps=inference_steps, generator=generator, height=height, width=width, guidance_scale=guidance_scale).images[0]
image

# Imagen a imagen

In [ ]:
%%capture
#@title 1- Descargar el modelo
from diffusers import StableDiffusionImg2ImgPipeline
from google.colab import files
from io import BytesIO
# Display images.
from IPython.display import display
from PIL import Image, ImageEnhance

device = "cuda"
model_path = "CompVis/stable-diffusion-v1-4"

pipe_img = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_path,
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
)
pipe_img = pipe_img.to(device)

In [ ]:
#@title 2- Seleccionar imagen inicial
import os
from google.colab import files
import shutil

upload_folder = 'inputs'
result_folder = 'results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

# upload files
uploaded = files.upload()

image_name = ""

for k in uploaded.keys():
  image_name = k

for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

img = Image.open('inputs/' + image_name)
  
width = img.width
height = img.height

size = (width, height)

image = Image.open(BytesIO(uploaded[image_name])).resize(size)
display(image)

In [ ]:
#@title 3- Generar la nueva imagen

prompt = "the portrait of a green robot" #@param {type:"string"}
inference_steps = 20 #@param {type:"number"}
guidance_scale = 7.5 #@param {type:"number"}
strength = 0.75 #@param {type:"number"}
#random_seed = True #@param {type:"boolean"}


#seed = 0 #@param {type:"number"}

#if random_seed:
#  seed = randint(0, 100000000)

generator2 = torch.Generator(device=device).manual_seed(234)

with autocast("cuda"):
    image = pipe_img(prompt=prompt, num_inference_steps=inference_steps, init_image=image, strength=strength, guidance_scale=guidance_scale, generator=generator2).images[0]
image

# Aumentar resolución

In [ ]:
%%capture
#@title 1- Instalar dependencias
!git clone https://github.com/sberbank-ai/Real-ESRGAN
%cd Real-ESRGAN
!pip install -r requirements.txt
# download model weights
# x2 
!gdown https://drive.google.com/uc?id=1pG2S3sYvSaO0V0B8QPOl1RapPHpUGOaV -O weights/RealESRGAN_x2.pth
# x4
!gdown https://drive.google.com/uc?id=1SGHdZAln4en65_NQeQY9UjchtkEF9f5F -O weights/RealESRGAN_x4.pth
# x8
!gdown https://drive.google.com/uc?id=1mT9ewx86PSrc43b-ax47l1E2UzR7Ln4j -O weights/RealESRGAN_x8.pth

In [23]:
#@title 2- Seleccionar imagen
import os
from google.colab import files
import shutil

upload_folder = 'inputs'
result_folder = 'results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

# upload files
uploaded = files.upload()

img_name = ""

for k in uploaded.keys():
  img_name = k

for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

Saving descarga (22).png to descarga (22).png
move descarga (22).png to inputs/descarga (22).png


In [ ]:
#@title 3- Aumentar resolución x4
from realesrgan import RealESRGAN
from PIL import Image
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model = RealESRGAN(device, scale=4)
model.load_weights('weights/RealESRGAN_x4.pth')

path_to_image = 'inputs/' + img_name
result_image_path = 'results/sr_image.png'

image = Image.open(path_to_image).convert('RGB')
sr_image = model.predict(np.array(image))

sr_image.save(result_image_path)
sr_image